# Lab 2: Probability, Statistics & Significance in Bioinformatics

This notebook follows the 90-minute lab exercises: from sampling and t-tests to BH FDR, bootstrapping, and a short Bayesian demo. The notebook will attempt to download a small RNA-seq dataset from GEO; if that fails (no internet / package), it will simulate a dataset.

**Environment:** Python, `numpy`, `pandas`, `scipy`, `statsmodels`, `matplotlib`, `seaborn`.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests

sns.set(style='whitegrid', context='notebook', rc={'figure.figsize':(8,5)})
np.random.seed(42)

print('Libraries imported. numpy, pandas, scipy, statsmodels, matplotlib, seaborn')

In [ ]:
# Try to fetch GEO dataset; fallback to simulated if unavailable
use_real = False
try:
    import GEOparse
    print('GEOparse available. Attempting to download a small dataset (this may fail if no internet).')
    gse_id = 'GSE10072'
    try:
        gse = GEOparse.get_GEO(geo=gse_id, destdir='.')
        
        # Try to get expression data from the correct location
        if hasattr(gse, 'pivot_samples') and callable(gse.pivot_samples):
            try:
                expr_table = gse.pivot_samples('VALUE')
                if expr_table is not None and not expr_table.empty:
                    use_real = True
                    print(f'Loaded expression table from {gse_id} (shape: {expr_table.shape})')
                else:
                    print('Expression table is empty; will fallback to simulation.')
            except Exception as pivot_e:
                print(f'Failed to pivot samples: {pivot_e}')
                # Try alternative method - access through samples
                if hasattr(gse, 'samples') and gse.samples:
                    sample_data = []
                    sample_names = []
                    for sample_id, sample in gse.samples.items():
                        if hasattr(sample, 'table') and sample.table is not None:
                            sample_data.append(sample.table['VALUE'].values)
                            sample_names.append(sample_id)
                    
                    if sample_data:
                        expr_table = pd.DataFrame(sample_data).T
                        expr_table.columns = sample_names
                        use_real = True
                        print(f'Loaded expression table from {gse_id} via samples (shape: {expr_table.shape})')
                    else:
                        print('No expression data found in samples; will fallback to simulation.')
                else:
                    print('No samples found; will fallback to simulation.')
        else:
            print('GSE object does not have pivot_samples method; will fallback to simulation.')
    except Exception as e:
        print('Failed to download or parse GEO series:', e)
except Exception as e:
    print('GEOparse not available or import failed:', e)

if not use_real:
    print('Falling back to simulated RNA-seq-like dataset (Negative Binomial counts).')
    n_genes = 100
    n_ctrl = 3
    n_treat = 3
    mu = np.random.gamma(shape=2.0, scale=10.0, size=n_genes)
    dispersion = 0.2
    counts_ctrl = np.vstack([np.random.negative_binomial(n=1/dispersion, p=(1/(1+mu[i]*dispersion)), size=n_ctrl)
                              for i in range(n_genes)])
    mu_treat = mu.copy()
    de_genes = np.random.choice(n_genes, size=10, replace=False)
    mu_treat[de_genes[:5]] *= 3.0
    mu_treat[de_genes[5:]] *= 0.4
    counts_treat = np.vstack([np.random.negative_binomial(n=1/dispersion, p=(1/(1+mu_treat[i]*dispersion)), size=n_treat)
                               for i in range(n_genes)])
    counts = np.hstack([counts_ctrl, counts_treat]).astype(int)
    sample_labels = ['ctrl1','ctrl2','ctrl3','trt1','trt2','trt3']
    expr_df = pd.DataFrame(counts, columns=sample_labels)
    expr_df.index = [f'gene_{i+1}' for i in range(n_genes)]
    print('Simulated count matrix shape:', expr_df.shape)
    expr_df.head()
else:
    # Process real GEO data
    print('Using real GEO data. Processing expression matrix...')
    # Clean up the data - remove any non-numeric columns and handle missing values
    expr_df = expr_table.select_dtypes(include=[np.number])
    expr_df = expr_df.fillna(0)  # Fill missing values with 0
    
    # If we have too many genes, subsample for computational efficiency
    if expr_df.shape[0] > 1000:
        print(f'Subsampling from {expr_df.shape[0]} to 1000 genes for efficiency')
        expr_df = expr_df.sample(n=1000, random_state=42)
    
    # Create artificial treatment groups for demonstration
    # In real analysis, you would use the actual sample metadata
    n_samples = expr_df.shape[1]
    n_ctrl = n_samples // 2
    n_treat = n_samples - n_ctrl
    
    # Rename columns to match the expected format
    ctrl_cols = expr_df.columns[:n_ctrl]
    treat_cols = expr_df.columns[n_ctrl:n_ctrl+n_treat]
    
    # Rename columns for consistency with the rest of the notebook
    new_cols = ['ctrl' + str(i+1) for i in range(n_ctrl)] + ['trt' + str(i+1) for i in range(n_treat)]
    expr_df.columns = new_cols
    
    print(f'Real data matrix shape: {expr_df.shape}')
    print('Sample groups:', new_cols)
    expr_df.head()

## Exercise 0 — Setup & Warm-up

Explore the dataset: view head, summary statistics, and plot histogram of expression for one gene.

In [ ]:
print('Expression matrix shape:', expr_df.shape)
expr_df.head()

expr_df.describe().T.head()

# Plot histogram of a single gene
gene = expr_df.index[0]

# TODO Plot a histogram of gene expression

## Exercise 1 — Probability & Sampling

In [ ]:
pop = np.random.normal(loc=10, scale=2, size=100000)
for n in [10,50,200]:
    means = [np.mean(np.random.choice(pop, size=n, replace=True)) for _ in range(2000)]
    sns.kdeplot(means, label=f'n={n}')
plt.legend()
plt.title('Sampling distribution of the mean for different sample sizes')
plt.show()

# Compare Poisson and NB
lam = 10
pois = np.random.poisson(lam, size=1000)
nb = np.random.negative_binomial(n=1/0.2, p=(1/(1+lam*0.2)), size=1000)

# TODO plot overlay of Poisson and NB distributions

## Exercise 2 — Hypothesis Testing: t-test

In [ ]:
gene = expr_df.index[0]
ctrl = expr_df.loc[gene, expr_df.columns.str.contains('ctrl')]
trt = expr_df.loc[gene, expr_df.columns.str.contains('trt')]
tstat, pval = stats.ttest_ind(ctrl, trt, equal_var=False)
print(f'Gene: {gene}, t-stat={tstat:.3f}, p-value={pval:.3g}')

# boxplot
import pandas as pd
data = pd.DataFrame({'count': np.concatenate([ctrl.values, trt.values]),
                     'group': ['ctrl']*len(ctrl) + ['trt']*len(trt)})

# TODO plot boxplot containing both ctrl and trt groups. What does the boxplot tell you?


## Exercise 3 — ANOVA

In [ ]:
g1 = np.random.normal(10,2,20)
g2 = np.random.normal(11,2,20)
g3 = np.random.normal(14,2,20)
fstat, pval_anova = stats.f_oneway(g1,g2,g3)
print(f'ANOVA F-stat={fstat:.3f}, p-value={pval_anova:.3g}')

df_anova = pd.DataFrame({'value': np.concatenate([g1,g2,g3]),
                         'group': ['g1']*len(g1) + ['g2']*len(g2) + ['g3']*len(g3)})
# TODO plot boxplot containing all three groups. What does the boxplot tell you?

## Exercise 4 — Multiple Testing & FDR

In [ ]:
pvals = []
log2fc = []
# Obtain p-values and log2FC for all genes
for g in expr_df.index:
    a = expr_df.loc[g, expr_df.columns.str.contains('ctrl')].values
    b = expr_df.loc[g, expr_df.columns.str.contains('trt')].values
    t, p = stats.ttest_ind(a, b, equal_var=False)
    pvals.append(p if not np.isnan(p) else 1.0)
    log2fc.append(np.log2((b.mean()+1)/(a.mean()+1)))

pvals = np.array(pvals)
log2fc = np.array(log2fc)
rej, pvals_corrected, alphacSidak, alphacBonf = multipletests(pvals, alpha=0.05, method='fdr_bh')
print(f"Raw significant (p<0.05): {(pvals<0.05).sum()} / {len(pvals)}")
print(f"BH-significant (FDR<0.05): {rej.sum()}")

# TODO plot volcano plot of all genes (scatter plot of log2FC vs -log10(p-value)). What does the volcano plot tell you?


## Exercise 5 — DGE Mini-Analysis

In [ ]:
res = pd.DataFrame({'gene': expr_df.index, 'log2FC': log2fc, 'pval': pvals, 'pval_adj': pvals_corrected, 'significant': rej}).set_index('gene').sort_values('pval')
res.head()
import seaborn as sns

# top genes
# TODO select top 10 upregulated and downregulated genes. What do you think about the results?
top_up = []
top_down = []

# Heatmap
# TODO plot heatmap of top candidate genes. What does the heatmap tell you?

## Exercise 6 — Bootstrapping Confidence Intervals

In [ ]:
gene = expr_df.index[0]
values = expr_df.loc[gene].values
n_boot = 2000
# TODO plot bootstrap distribution of mean for the first gene (sample randomly with replacement from the gene's mean expression values, and then calculate Confidence Interval [2.5,97.5]). 


## Exercise 7 — Bayesian Inference Basics (Simple Demo)

In [ ]:
from scipy.stats import beta
# Let's say we have a coin that we know is fair (p=0.5). We flip it 6 times and get 4 heads. What is the posterior distribution of the probability of heads?
alpha_prior, beta_prior = 1, 1
successes = 4
trials = 6
# The coin toss may be biased, so we update our prior belief about the probability of heads.
# The coin toss may be modelled by a binomial distribution, which is a special case of the beta-binomial distribution.
# The beta-binomial distribution is a conjugate prior for the binomial distribution.
# The posterior distribution is given by the beta distribution.
alpha_post = alpha_prior + successes
beta_post = beta_prior + (trials - successes)
x = np.linspace(0,1,200)

# TODO plot prior and posterior distributions. What does the posterior distribution tell you?


## Exercise 8 — Model Confidence in ML (Optional)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Demonstration of a simple model confidence interval

X = expr_df.T.mean(axis=1).values.reshape(-1,1)
y = np.array([0]*3 + [1]*3)
accs = []
for _ in range(200):
    idx = np.random.choice(len(y), size=len(y), replace=True)
    Xb, yb = X[idx], y[idx]
    clf = LogisticRegression().fit(Xb, yb)
    preds = clf.predict(Xb)
    accs.append(accuracy_score(yb, preds))
np.percentile(accs, [2.5,50,97.5])